# Recomendador de perfis
---
Uma vez computado o índice contendos os K vizinhos mais próximos de cada um dos 85.000+ usuários e a lista de atribuição dos perfis, um recomendador de perfis será construído baseado em:

1 - A vizinhança de um usuário

2 - Os perfis que a vizinhança possui

3 - Os perfis que o usuário não possui

4 - As chamadas atribuições categóricas dos perfis ainda não atribuídos ao usuário

In [1]:
import os
import numpy as np
import pandas as pd
import pynndescent
import duckdb
import pickle
import scipy.sparse
from knn_indexer import KnnIndex

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

As variáveis de ambiente abaixo precisam ser configuradas antes da execução deste notebook. Vide o arquivo **setenv.ps1.example**

In [3]:
DATASET                = os.environ['DATASET']
USER_ROLES             = os.environ['USER_ROLES']
ORGUNIT_ROLES          = os.environ['ORGUNIT_ROLES']
FUNCTION_ROLES         = os.environ['FUNCTION_ROLES']
KNN_INDEX              = os.environ['KNN_INDEX']
HASHED_FEATURES_IDX    = os.environ['HASHED_FEATURES_IDX']
CUTOFF_DISTANCE        = 1.0

Leitura do arquivo contendos os dados do usuário e os perfis já atribuídos

In [4]:
dataset_df = pd.read_parquet(DATASET)
dataset_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86332 entries, 0 to 86331
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   chave_usuario            86300 non-null  object
 1   tipo_usuario             86332 non-null  object
 2   centro_custo             39577 non-null  object
 3   lotacao_topo             86332 non-null  object
 4   sigla_lotacao            86332 non-null  object
 5   nome_lotacao             86332 non-null  object
 6   cargo                    86332 non-null  object
 7   enfase                   86332 non-null  object
 8   funcao                   86332 non-null  object
 9   sindicato                39070 non-null  object
 10  area_rh                  85902 non-null  object
 11  imovel                   85902 non-null  object
 12  local_negocio            85900 non-null  object
 13  grupo_prestacao_servico  46192 non-null  object
 14  regime_trabalho          73068 non-nul

In [5]:
user_roles_df = pd.read_parquet(USER_ROLES)
user_roles_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411381 entries, 0 to 411380
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   chave_usuario  411381 non-null  object
 1   role           411381 non-null  object
dtypes: object(2)
memory usage: 6.3+ MB


Leitura do arquivo contendo as atribuições dos perfis as diferentes lotações

In [6]:
orgunit_roles_df = pd.read_parquet(ORGUNIT_ROLES)
orgunit_roles_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28171 entries, 0 to 28170
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   lotacao_topo   28171 non-null  object
 1   sigla_lotacao  28171 non-null  object
 2   role           28171 non-null  object
 3   atribuicoes    28171 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 880.5+ KB


Leitura do arquivo contendo as atriabuições do perfil para os tipos de usuário, cargos, ênfases e funções gratificadas

In [7]:
function_roles_df = pd.read_parquet(FUNCTION_ROLES)
function_roles_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13865 entries, 0 to 13864
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tipo_usuario  13865 non-null  object
 1   cargo         13865 non-null  object
 2   enfase        13865 non-null  object
 3   funcao        13865 non-null  object
 4   role          13865 non-null  object
 5   atribuicoes   13865 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 650.0+ KB


Leitura do índice pré-computado de distâncias entre os 85.000+ usuários

In [8]:
knn_index = KnnIndex.load(KNN_INDEX)
knn_index.get_neighbors(0)

(array([11080,     0, 59152, 51250, 24581, 24742, 58253,  5381, 29334,
        19220, 81875,  5129, 64794, 59437, 28229,   967, 80180, 58105,
        27382, 26544, 58198, 38320, 27953, 56639, 10148, 14332, 25573,
          908, 26130, 64954, 25287, 58041, 26495, 15523, 28842, 26753,
         5354, 63230,  5418,  9597, 32426, 57225, 28978, 32496, 13568,
         8499, 19391, 25197, 71572, 14190, 33207,  3496, 33537, 52526,
        33525, 33464, 58827,  8329, 53906, 30310, 26543,  1146, 31718,
         5602, 25909, 32453, 27874,  1514, 80161, 63641, 33206, 18867,
        27564, 27567, 54165, 23454, 12672, 27360,  1602,  7567, 18159,
        38197, 68702, 26431, 25670, 65608, 58165, 29158, 24517,  2119,
         2101,  8208, 54040, 25559, 33876, 38327, 80129, 38239, 11533,
         2113, 26032, 17764, 31191,  2108, 81978, 26091, 71450, 37677,
        15539, 85250, 73627, 58949, 54850,  7533, 32389,  2780, 45425,
        45592, 25760,  2102, 16067,  2791,  7597, 36230, 45589, 27496,
      

Leitura do índice contendo os dados dos usuários para uso da matriz esparsa de features por usuário

In [9]:
hashed_features_idx = pd.read_parquet(HASHED_FEATURES_IDX)
hashed_features_idx.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86332 entries, 0 to 86331
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          86332 non-null  int64 
 1   chave_usuario  86300 non-null  object
 2   lotacao_topo   86332 non-null  object
 3   sigla_lotacao  86332 non-null  object
 4   tipo_usuario   86332 non-null  object
 5   cargo          86332 non-null  object
 6   enfase         86332 non-null  object
 7   funcao         86332 non-null  object
dtypes: int64(1), object(7)
memory usage: 5.3+ MB


O trecho de código abaixo mostra como realizar uma busca no índice de usuários para recuperar a posição em que um usuário específico situa-se na matriz esparsa numérica e com isso recuperar sua vizinhança bem como as distâncias calculadas de seus vizinhos.

Um DataFrame é construído possibilitando a exibição destes dados e seu uso em conjunto com a biblioteca **DuckDB**

In [10]:
search_user   = "U4UL"
search_df     = hashed_features_idx[hashed_features_idx["chave_usuario"] == search_user].copy()

print(f"Usuário '{search_user}' encontrado na massa de perfis analisada")
search_index   = search_df.iloc[0]["index"]
indices, distances = knn_index.get_neighbors(search_index)
_neighbors_df = pd.DataFrame({"index": indices, "distance": distances})

display(search_df)
display(_neighbors_df)

Usuário 'U4UL' encontrado na massa de perfis analisada


,index,chave_usuario,lotacao_topo,sigla_lotacao,tipo_usuario,cargo,enfase,funcao
75984,75984,U4UL,TIC,TIC/CORP/DSCESI/DS-PDDS,EMPREGADO,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ANALISE DE SISTEMAS ENG SOFTWARE,n/a


,index,distance
0,75984,2.384185e-07
1,78207,6.878667e-02
2,83306,6.878667e-02
3,78248,6.878667e-02
4,81704,1.025241e-01
5,81703,1.025241e-01
6,81674,1.025241e-01
7,77620,1.033726e-01
8,52373,1.033726e-01
9,24044,1.057565e-01


A query abaixo é utilizada para retornar a vizinhança de um usuário e, para cada vizinho, a listagem de perfis atribuídos à cada vizinho que ainda não foram atribuídos ao usuário.

As chamadas atribuições categóricas são também computadas para as roles dos vizinhos.


 - atribuido_tipo_usuario - O perfil possui atribuição ao mesmo tipo de usuário
 
 - atribuido_topo - O perfil possui atribuilção à mesma lotação topo do usuário
 
 - atribuido_lotacao - O perfil possui atribuição à mesma lotação do usuário
 
 - atribuido_cargo - O perfil possuí atribuição ao mesmo cargo do usuário
 
 - atribuido_enfase - O perfil possui atribuição à mesma ênfase do usuário
 
 - atribuido_funcao - O perfil possui atribuição à mesma função gratificada do usuário
 
 
Em todos os casos acima, as atribuições categóricas dizem respeito à perfis ainda não atribuídos ao próprio usuário. Estas informações figuram de forma proeminente no cômputo do score da recomendação conforme abaixo.

A consulta é executada 50 vezes para que qualquer regressão de performance na query seja imediatamente observada.

In [11]:
conn = duckdb.connect(":memory:")

In [12]:
sql = """
WITH cte_atribuido_tipo_usuario AS (
    SELECT DISTINCT tipo_usuario, role FROM function_roles_df
)
, cte_atribuido_topo AS (
    SELECT DISTINCT lotacao_topo, role FROM orgunit_roles_df
)
, cte_atribuido_lotacao AS (
    SELECT DISTINCT sigla_lotacao, role FROM orgunit_roles_df
)
, cte_atribuido_cargo AS (
    SELECT DISTINCT cargo, role FROM function_roles_df
)
, cte_atribuido_enfase AS (
    SELECT DISTINCT enfase, role FROM function_roles_df
)
, cte_atribuido_funcao AS (
    SELECT DISTINCT funcao, role FROM function_roles_df
)
SELECT  DISTINCT z.chave_usuario as usuario_busca
,       a.distance
,       b.chave_usuario
,       b.sigla_lotacao
,       b.cargo
,       b.enfase
,       b.funcao
,       c.role
,       CASE WHEN e.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_tipo_usuario
,       CASE WHEN f.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_topo        
,       CASE WHEN g.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_lotacao
,       CASE WHEN h.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_cargo
,       CASE WHEN i.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_enfase
,       CASE WHEN j.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_funcao
FROM    search_df z
        --
        CROSS JOIN _neighbors_df a
        --
        INNER JOIN hashed_features_idx b
        ON a.index              = b.index
        --
        INNER JOIN user_roles_df c
        ON b.chave_usuario      = c.chave_usuario
        --
        LEFT OUTER JOIN user_roles_df d
        ON  z.chave_usuario     = d.chave_usuario
        AND c.role              = d.role
        --
        LEFT OUTER JOIN cte_atribuido_tipo_usuario e
        ON  z.tipo_usuario      = e.tipo_usuario 
        AND c.role              = e.role
        --
        LEFT OUTER JOIN cte_atribuido_topo f
        ON  z.lotacao_topo      = f.lotacao_topo 
        AND c.role              = f.role
        --
        LEFT OUTER JOIN cte_atribuido_lotacao g
        ON  z.sigla_lotacao     = g.sigla_lotacao 
        AND c.role              = g.role
        --
        LEFT OUTER JOIN cte_atribuido_cargo h
        ON  z.cargo             = h.cargo 
        AND c.role              = h.role
        --
        LEFT OUTER JOIN cte_atribuido_enfase i
        ON  z.enfase            = i.enfase 
        AND c.role              = i.role
        --
        LEFT OUTER JOIN cte_atribuido_funcao j
        ON  z.funcao            = j.funcao 
        AND c.role              = j.role
        --
WHERE   b.chave_usuario         <> z.chave_usuario
AND     z.chave_usuario         IS NOT NULL
AND     d.role                  IS NULL
ORDER   BY a.distance
"""
for i in range(50): # to catch performance regressions
    neighbors_df = conn.execute(sql).fetchdf()
neighbors_df

,usuario_busca,distance,chave_usuario,sigla_lotacao,cargo,enfase,funcao,role,atribuido_tipo_usuario,atribuido_topo,atribuido_lotacao,atribuido_cargo,atribuido_enfase,atribuido_funcao
0,U4UL,0.132842,UPOT,TIC/CORP/DSCESI/DS-PDDS,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ANALISE DE SISTEMAS ENG SOFTWARE,n/a,Z:MM_PB001_CONSULTANTE_INF_MES,1,1,1,1,1,1
1,U4UL,0.132842,UPOT,TIC/CORP/DSCESI/DS-PDDS,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ANALISE DE SISTEMAS ENG SOFTWARE,n/a,Z:MM_PB001_CONS_COCKPIT,1,1,1,1,0,1
2,U4UL,0.163098,X0BY,TIC/CORP/RJTI/PN-STSMS,PROF. PETROBRAS DE NIVEL SUPERIOR MASTER,PCR NS ANALISE DE SISTEMAS ENG SOFTWARE,n/a,Z:PS_PBD06_APONTADOR_HORAS,1,0,0,1,1,1
3,U4UL,0.163873,UR5J,TIC/CORP/FRIE,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ANALISE DE SISTEMAS ENG SOFTWARE,n/a,Z:MM_PBAUT_FISCAL_CONTRATO,1,1,0,1,1,1
4,U4UL,0.163873,UR5J,TIC/CORP/FRIE,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ANALISE DE SISTEMAS ENG SOFTWARE,n/a,Z:FI_FT_PB001_SUP_PC_TIMP,0,0,0,0,0,0
5,U4UL,0.171267,UP2H,TIC/CORP/DSCESI/DS-PDDS,PROF. PETROBRAS DE NIVEL SUPERIOR PLENO,PCR NS ANALISE DE SISTEMAS ENG SOFTWARE,n/a,Z:BC_USO_GERAL,1,1,1,1,1,1
6,U4UL,0.172037,U4VE,TIC/CORP/DSCESI/DS-TEC,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ANALISE DE SISTEMAS ENG SOFTWARE,n/a,Z:BC_USO_GERAL,1,1,1,1,1,1
7,U4UL,0.174393,UP2Y,TIC/CORP/RJTI/PN-CRJ,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ANALISE DE SISTEMAS ENG SOFTWARE,n/a,Z:BC_USO_GERAL,1,1,1,1,1,1
8,U4UL,0.174393,UPN0,TIC/CORP/FRIE/PN-SDE,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ANALISE DE SISTEMAS ENG SOFTWARE,n/a,Z:BC_DESENVOLVEDOR_PERFORMANCE,1,1,0,0,1,1
9,U4UL,0.174461,CY57,TIC/CORP/PDGC/PN-RH,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ANALISE DE SISTEMAS ENG SOFTWARE,n/a,Z:BC_USO_GERAL,1,1,1,1,1,1


A query abaixo não é utilizada pelo recomendador mas ilustra as distâncias e quantitativos de usuários e perfis por gerências

In [13]:
sql = f"""
    SELECT  a.sigla_lotacao
    ,       MIN(distance)                   AS min_distance
    ,       AVG(distance)                   AS avg_distance
    ,       MAX(distance)                   AS max_distance
    ,       COUNT(DISTINCT a.chave_usuario) AS qtd_usuarios
    ,       COUNT(DISTINCT a.role)          AS qtd_roles
    FROM    neighbors_df a
    GROUP   BY a.sigla_lotacao
    --
    UNION
    --
    SELECT  '<todas lotações>'              AS sigla_lotacao
    ,       MIN(distance)                   AS min_distance
    ,       AVG(distance)                   AS avg_distance
    ,       MAX(distance)                   AS max_distance
    ,       COUNT(DISTINCT a.chave_usuario) AS qtd_usuarios
    ,       COUNT(DISTINCT a.role) AS qtd_roles
    FROM    neighbors_df a
    --
    ORDER   BY qtd_roles DESC
"""
for i in range(50): # to catch performance regressions
    lotacoes_vizinhas_df = conn.execute(sql).fetchdf()
lotacoes_vizinhas_df

,sigla_lotacao,min_distance,avg_distance,max_distance,qtd_usuarios,qtd_roles
0,<todas lotações>,0.132842,0.235844,0.274046,72,39
1,TIC/CORP/FRIE/PN-SDE,0.174393,0.260091,0.271500,4,10
2,TIC/CORP/DSCESI/DS-PDDS,0.132842,0.192393,0.265683,4,9
3,TIC/CORP/DSCESI/DS-SIG,0.240772,0.262750,0.266413,5,5
4,TIC/CORP/FRIE,0.163873,0.236520,0.269211,5,4
5,TIC/CORP/PDGC/PN-RH,0.174461,0.203602,0.243165,5,4
6,TIC/CORP/FRIE/PN-FRS,0.202262,0.222686,0.243109,3,4
7,TIC/RGNCL/RGN/PN-GEINP,0.203038,0.239169,0.251213,2,4
8,TIC/US/DP/PN-SRGE-GIRP-PDP,0.227982,0.248581,0.269180,2,4
9,TIC/CORP/RJTI/PN-CI,0.202318,0.215934,0.243165,2,3


A query abaixo retorna os perfis recomendados com seus respectivos scores calculados.

**TODO: Inserir notação matemática da fórmula do score aqui como um imagem**

Query executada 50 vezes para capturar possíveis regressões de performance

In [14]:
sql = """
WITH cte_roles AS (
    SELECT  a.role
    ,       MAX(atribuido_tipo_usuario)     AS atribuido_tipo_usuario
    ,       MAX(atribuido_topo)             AS atribuido_topo
    ,       MAX(atribuido_lotacao)          AS atribuido_lotacao
    ,       MAX(atribuido_cargo)            AS atribuido_cargo
    ,       MAX(atribuido_enfase)           AS atribuido_enfase
    ,       MAX(atribuido_funcao)           AS atribuido_funcao
    ,       MAX(atribuido_tipo_usuario)
    +       MAX(atribuido_topo)
    +       MAX(atribuido_lotacao)
    +       MAX(atribuido_cargo)
    +       MAX(atribuido_enfase)
    +       MAX(atribuido_funcao)           AS atribuicoes_categoricas
    ,       MIN(distance)                   AS min_distance
    ,       AVG(distance)                   AS avg_distance
    ,       ( (1 - MIN(distance)) + AVG(distance)) * 0.5 AS distance_factor     
    ,       COUNT()                         AS qtd_atribuicoes
    FROM    neighbors_df a
    WHERE   a.distance                      <= ?
    GROUP   BY a.role
)
, cte_role_scores AS (
    SELECT  a.role
    ,       ( a.atribuicoes_categoricas * a.qtd_atribuicoes * a.distance_factor )
    /       ( (SELECT MAX(atribuicoes_categoricas) FROM cte_roles) 
    *         (SELECT MAX(qtd_atribuicoes)         FROM cte_roles)
    *         (SELECT MAX(distance_factor)         FROM cte_roles)
            ) AS score   
    FROM    cte_roles a
)
SELECT  a.role
,       a.atribuicoes_categoricas
,       a.min_distance
,       a.avg_distance
,       a.distance_factor
,       a.qtd_atribuicoes
,       b.score
,       a.atribuido_tipo_usuario
,       a.atribuido_topo
,       a.atribuido_lotacao
,       a.atribuido_cargo
,       a.atribuido_enfase
,       a.atribuido_funcao
FROM    cte_roles a
        --
        INNER JOIN cte_role_scores b
        ON  a.role                      = b.role
        --
WHERE   b.score                         > 0.0
ORDER   BY b.score DESC
"""
params = (CUTOFF_DISTANCE,)
for i in range(50): # to catch performance regressions
    recomendations_df = conn.execute(sql, params).fetchdf()
recomendations_df

,role,atribuicoes_categoricas,min_distance,avg_distance,distance_factor,qtd_atribuicoes,score,atribuido_tipo_usuario,atribuido_topo,atribuido_lotacao,atribuido_cargo,atribuido_enfase,atribuido_funcao
0,Z:BC_USO_GERAL,6,0.171267,0.236727,0.532730,30,0.992448,1,1,1,1,1,1
1,Z:MM_PBAUT_FISCAL_CONTRATO,5,0.163873,0.237440,0.536784,10,0.277778,1,1,0,1,1,1
2,Z:BC_CUTOVER_FIORI,5,0.202262,0.242436,0.520087,7,0.188396,1,1,0,1,1,1
3,Z:BC_CUTOVER_S4,5,0.202262,0.242436,0.520087,7,0.188396,1,1,0,1,1,1
4,Z:PS_PB003_APONTADOR_HORAS,5,0.188604,0.229549,0.520472,6,0.161602,1,1,0,1,1,1
5,Z:BC_CHARM_GMUD,5,0.203686,0.239383,0.517848,6,0.160787,1,1,0,1,1,1
6,Z:MM_ARIBA_GERAL,5,0.202262,0.229623,0.513681,4,0.106329,1,1,0,1,1,1
7,Z:PS_PBD06_APONTADOR_HORAS,4,0.163098,0.211524,0.524213,4,0.086807,1,0,0,1,1,1
8,Z:MM_PB001_CONSULTANTE_INF_MES,6,0.132842,0.180144,0.523651,2,0.065036,1,1,1,1,1,1
9,Z:MM_PB001_LIBERADOR_NL_PGTO,5,0.266413,0.267499,0.500543,2,0.051805,1,1,0,1,1,1
